In [ ]:
# Run in python 3.7+
# Script for all sensor data


import arable
from   arable.client   import ArableClient
import re
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from   datetime        import timedelta
from   datetime        import datetime
import pandas          as     pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from   io              import StringIO
from   sensor_includes import email, password_ccber, tenant_ccber

In [ ]:
def change(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
def changeback(x):
    return datetime.strptime(x, "%Y-%m-%d")
def la(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")
def tests(x):
    return x.strftime("%Y-%m")
def ha(a, b):
    a[0].append(change(b))
    a[1].append(tests(b))
    a[2].append(str(b))
    a[3].append(tests(b))
def haha(a, b):
    a[0].append(str(b))
    a[1].append(str(tests(b)))
def hahaha(a, b):
    for i in range(len(a)):
        a[i].append(b[i])
def hahahaha(a, b):
    a[2].append(str(b))
    a[3].append(tests(b))

In [ ]:
a = ArableClient()
a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

In [ ]:
device_ids     = [] # Ex. A00****
device_names   = [] # Ex. Lagoon Ice Plants
device_created = [] # Ex. 2019-10-10T00:00:00

# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_names.
# The corresponding Device ID is added to device_ids.
for i in a.devices():
    keys = list(i.keys())
    for j in range(len(keys)):
        if str(keys[j]).startswith('location'): 
            values = list(i.values())
            loc = str(values[j])
            loc_name = (re.split(r':', loc)[2])[2:-2]
            if re.search("Z", loc_name):
                device_ids.append(str(values[j+2]))
                device_names.append(loc_name)
                device_created.append(str(values[j-28])[:-7])

In [ ]:
dt      = datetime.now()

dates    = [la(i) for i in device_created]
print(device_ids)

levels = np.tile([6, -4,  3, -3, 2, -8, 7, -5, 6,  -5, 1, -1],
                 int(np.ceil(len(dates)/12)))[:len(dates)]

fig, ax = plt.subplots(figsize=(10, 3))
ax.set(title="Sensor Startup Date")

markerline, stemline, baseline = ax.stem(dates, levels,
                                         linefmt="k-", basefmt="k-",
                                         use_line_collection=True)

plt.setp(markerline, mec="k", mfc="w", zorder=3)
markerline.set_ydata(np.zeros(len(dates)))

vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
for d, l, r, va in zip(dates, levels, device_ids, vert):
    ax.annotate(r, xy=(d, l), xytext=(-3, np.sign(l)*3),
                textcoords="offset points", va=va, ha="right")

# format xaxis with 4 month intervals
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%d %b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# remove y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.5)
plt.show()

In [ ]:
# Converts each item in device_ids to a list.          
device_ids = list(map(lambda el:[el], device_ids))

In [ ]:
latitude  = []
longitude = []
for i in device_ids:
    #print(i)
    df = a.query(select = 'all', 
                 format = 'csv', 
                 devices = i, 
                 measure = "daily", 
                 order = "time", 
                 end = change(dt), 
                 start = change(dt - timedelta(days=1)), 
                 limit = 100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    latitude.append(float(str(df['lat'].values.astype(float)).strip('[ ]')))
    longitude.append(float(str(df['long'].values.astype(float)).strip('[ ]')))
print(latitude)
print(longitude)

d = {'name': device_ids, 'lat': latitude, 'lon': longitude}
dd = pd.DataFrame(data=d)
print(dd)

In [ ]:
import altair as alt
from vega_datasets import data

#states = alt.topo_feature(data.us_10m.url, feature='states')

# Load the data, which is loaded as a dict object
us_10m  = data.us_10m()
# Select the geometries under states under objects, filter on id (9,25,34,36,42,44)
us_10m['objects']['states']['geometries']=[item for item in us_10m['objects'] \
      ['states']['geometries'] if item['id'] in [6]]
# Make the topojson data
states = alt.Data(
    values=us_10m, 
    format=alt.TopoDataFormat(feature='states',type='topojson'))


# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

points = alt.Chart(dd).mark_point(
    size=100,
    color='black'
).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    tooltip='name:O'
)


background + points

In [ ]:
fig, ax = plt.subplots()
ax.scatter(latitude, longitude, marker = 'o')
plt.title('Sensor Location')
plt.ylabel('longitude')
plt.ylim(-119.844, -119.85)
plt.xlim(34.410, 34.405)
plt.xlabel('latitude')

for i, txt in enumerate(device_names):
    ax.annotate(txt[1:],
                (latitude[i], longitude[i]),
                xytext=(-(latitude[i])*0.2*(i+1),-(latitude[i])*0.2*(i+1)),
                xycoords="data",
                textcoords="offset points", 
                arrowprops=dict(arrowstyle='->'), 
                va="bottom",
                ha="left")

plt.show()

In [ ]:
# This can be deleted later.
s_month = []
s_year  = []
for i in device_created:
    s_month.append(int(i[5:-19]))
    s_year.append(int(i[:-22]))

In [ ]:
sta_m   = []
sta_m_f = []
end_m   = []
end_m_f = []
mm      = [sta_m, sta_m_f, end_m, end_m_f]

sta_y   = []
sta_y_f = []
end_y   = []
end_y_f = []
yy      = [sta_y, sta_y_f, end_y, end_y_f]

In [ ]:
for j in range(len(device_created)):
    sta_m_t   = []
    sta_m_f_t = []
    end_m_t   = []
    end_m_f_t = []
    sta_y_t   = []
    sta_y_f_t = []
    end_y_t   = []
    end_y_f_t = []
    mo = [sta_m_t, sta_m_f_t, end_m_t, end_m_f_t]
    ye = [sta_y_t, sta_y_f_t, end_y_t, end_y_f_t]
    s  = changeback((device_created[j])[:10])
    haha(mo, s)
    haha(ye, s)
    s  = s.replace(day=1)
    if s.year == dt.year:
        for i in range(s.month+1, dt.month+1):
            ha(mo, s.replace(month=i))
    elif s.year != dt.year: 
        for i in range(dt.year-s.year):
            for j in range(s.month+1, 13):
                ha(mo, s.replace(month=j))
        for i in range(s.year+1, dt.year+1):
            x = s.replace(year=i, month=1)
            ha(ye, x)
            for j in range(1, dt.month+1):
                ha(mo, x.replace(month=j)) 
    hahahaha(mo, dt)
    hahahaha(ye, dt)
    hahaha(mm, mo)
    hahaha(yy, ye)
print(sta_m)

In [ ]:
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
def test(sta, end, i, sta_f, end_f, device):
    for j in range(len(sta)):
        try:
            df = a.query(select = 'all', format = 'csv', devices = device, measure = str(i), order = "time", end = end[j], start = sta[j], limit = 100000000)
            fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
            print(fn)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            df = df.drop('location', 1)
            df.to_csv(fn, sep = ",")
        except:
            print('ERROR WRITING FILE')
            continue

In [ ]:
for i in range(len(device_ids)):
    test(sta_m[i], end_m[i], "hourly", sta_m_f[i], end_m_f[i], device_ids[i])
    test(sta_y[i], end_y[i], "daily", sta_y_f[i], end_y_f[i], device_ids[i])

In [ ]:
de = a.query(select = 'all', 
             format = 'csv', 
             devices = device_ids[0], 
             measure = "hourly", 
             order = "time", 
             end = (end_m[0])[1], 
             start = (sta_m[0])[1], 
             limit = 100000000)
de = StringIO(de)
de = pd.read_csv(de, sep=',', error_bad_lines=False)

de['time'] = pd.to_datetime(de['time'],infer_datetime_format=True)

In [ ]:
base = alt.Chart(de, title=str((device_names[0])[1:]) + ' \n ' + str(((sta_m[0])[0])[:10]) + ' -- ' + str((end_m[0])[0])[:10]
).mark_line(
    color='black'
).properties(
    width=800,
    height=100
).encode(
    x='time:T',
    y='Tabove:Q')

s = alt.layer( *[base.mark_line(color='blue').encode(y='Tbelow')], data=de)
z = alt.layer( *[base.mark_line(color='red').encode(y='Tair')], data=de)
q = alt.layer( *[base.mark_line(color='steelblue').encode(y='Tdew')], data=de)
base + s + z + q

In [ ]:
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)

fig, ax = plt.subplots(figsize=(3, 3))
ax = plt.gca()

df.plot(kind='line',x='time',y='Tabove',ax=ax)
df.plot(kind='line',x='time',y='Tbelow', color='red', ax=ax)
df.plot(kind='line',x='time',y='Tair', color='green', ax=ax)
df.plot(kind='line',x='time',y='Tdew', color='blue', ax=ax)
plt.xlabel('')
plt.ylabel('Deg. C')
plt.title(str((device_names[0])[1:]) + ' \n ' + str(((sta_m[0])[0])[:10]) + ' -- ' + str((end_m[0])[0])[:10])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()